home directory: cluster_doctor
function availability : ./kubectl/
1. get free nodes list
    - save it to list ./gitignored/free.txt
2. Get latest test results timmestamp from validation.db for all the free nodes in the free node list ( by accessing gcr-admin-pvc-access pod)
    - per node per test - latest timestamp
    - if a node has no test results - mark it as "never tested" - highest priority
3. Combine free nodes list with latest test results timestamps create a priority queue
    - Sort nodes by latest test results timestamps (oldest first - highest priority)
    - If the test results are not older than Z days - skip that node from the priority Queue
4. For each node in job queue list
    - Create a job to run cluster-doctor validation tests on that node
    - Submit the job to k8s cluster
    - Log job submission status

5. Monitor job status
    - if a job pending for more than X minutes - cancel the job and log the event
    - if a job runs longer than Y minutes - cancel the job
6. After job completion ( inside job pod)
    - Collect test results log
    - Update validation.db with new test results and timestamps
    - Log job completion status

7. Generate a daily report
    - Summary of nodes tested
    - Summary of test results
    - List of nodes that were never tested
    - Save report to ./gitignored/reports/daily_report_<date>.txt

In [ ]:
# home directory: cluster_doctor
# function availability : ./kubectl/
# 1. get free nodes list
#     - save it to list ./gitignored/free.txt
# 2. Get latest test results timmestamp from validation.db for all the free nodes in the free node list ( by accessing gcr-admin-pvc-access pod)
#     - per node per test - latest timestamp
#     - if a node has no test results - mark it as "never tested" - highest priority
# 3. Combine free nodes list with latest test results timestamps create a priority queue
#     - Sort nodes by latest test results timestamps (oldest first - highest priority)
#     - If the test results are not older than Z days - skip that node from the priority Queue
# 4. For each node in job queue list
#     - Create a job to run cluster-doctor validation tests on that node
#     - Submit the job to k8s cluster
#     - Log job submission status

# 5. Monitor job status
#     - if a job pending for more than X minutes - cancel the job and log the event
#     - if a job runs longer than Y minutes - cancel the job
# 6. After job completion ( inside job pod)
#     - Collect test results log
#     - Update validation.db with new test results and timestamps
#     - Log job completion status

# 7. Generate a daily report
#     - Summary of nodes tested
#     - Summary of test results
#     - List of nodes that were never tested
#     - Save report to ./gitignored/reports/daily_report_<date>.txt


# variables
from symtable import Class
# import re # No longer needed

home_dir="/home/hari/b200/validation/cluster_doctor/"
functions_dir=home_dir+"kubectl/"


#Get free nodes list
# /home/hari/b200/validation/cluster_doctor/kubectl/cluster/freenode_list.sh

#create a cluster class

class Cluster:
    def __init__(self, ns):
        self.ns = ns
        #numerical timestamp
        self.timestamp = int(__import__('time').time())
        self.freenode_list = []

    def freenodes(self):
        import subprocess
        # Using the new script that returns just the node names
        cmd = functions_dir+"cluster/freenode_list.sh"
        result = subprocess.run([cmd], stdout=subprocess.PIPE, shell=True)
        # Split by newline and filter empty strings
        self.freenode_list = [n for n in result.stdout.decode('utf-8').strip().split('\n') if n]
        return self.freenode_list
    
    
    def latest_test_results(self, node):
        import subprocess
        cmd = functions_dir+"results/
        return output

SyntaxError: invalid syntax (3472657210.py, line 30)

In [10]:
cluster=Cluster("gcr-admin")
print(cluster.freenodes())

['slc01-cl02-hgx-0001', 'slc01-cl02-hgx-0006', 'slc01-cl02-hgx-0008', 'slc01-cl02-hgx-0009', 'slc01-cl02-hgx-0012', 'slc01-cl02-hgx-0013', 'slc01-cl02-hgx-0016', 'slc01-cl02-hgx-0020', 'slc01-cl02-hgx-0021', 'slc01-cl02-hgx-0022', 'slc01-cl02-hgx-0023', 'slc01-cl02-hgx-0024', 'slc01-cl02-hgx-0027', 'slc01-cl02-hgx-0029', 'slc01-cl02-hgx-0031', 'slc01-cl02-hgx-0034', 'slc01-cl02-hgx-0035', 'slc01-cl02-hgx-0047', 'slc01-cl02-hgx-0048', 'slc01-cl02-hgx-0050', 'slc01-cl02-hgx-0051', 'slc01-cl02-hgx-0054', 'slc01-cl02-hgx-0057', 'slc01-cl02-hgx-0062', 'slc01-cl02-hgx-0066', 'slc01-cl02-hgx-0072', 'slc01-cl02-hgx-0073', 'slc01-cl02-hgx-0074', 'slc01-cl02-hgx-0079', 'slc01-cl02-hgx-0082', 'slc01-cl02-hgx-0083', 'slc01-cl02-hgx-0085', 'slc01-cl02-hgx-0090', 'slc01-cl02-hgx-0092', 'slc01-cl02-hgx-0099', 'slc01-cl02-hgx-0102', 'slc01-cl02-hgx-0104', 'slc01-cl02-hgx-0107', 'slc01-cl02-hgx-0112', 'slc01-cl02-hgx-0115', 'slc01-cl02-hgx-0118', 'slc01-cl02-hgx-0119', 'slc01-cl02-hgx-0130', 'slc01-cl0

In [17]:
print(cluster.freenode_list)
print(cluster.timestamp)
# convert timestamp to human readable format
import datetime
readable_time = datetime.datetime.fromtimestamp(cluster.timestamp).strftime('%Y-%m-%d %H:%M:%S')
print("Cluster object created at:", readable_time)

['slc01-cl02-hgx-0001', 'slc01-cl02-hgx-0006', 'slc01-cl02-hgx-0008', 'slc01-cl02-hgx-0009', 'slc01-cl02-hgx-0012', 'slc01-cl02-hgx-0013', 'slc01-cl02-hgx-0016', 'slc01-cl02-hgx-0020', 'slc01-cl02-hgx-0021', 'slc01-cl02-hgx-0022', 'slc01-cl02-hgx-0023', 'slc01-cl02-hgx-0024', 'slc01-cl02-hgx-0027', 'slc01-cl02-hgx-0029', 'slc01-cl02-hgx-0031', 'slc01-cl02-hgx-0034', 'slc01-cl02-hgx-0035', 'slc01-cl02-hgx-0047', 'slc01-cl02-hgx-0048', 'slc01-cl02-hgx-0050', 'slc01-cl02-hgx-0051', 'slc01-cl02-hgx-0054', 'slc01-cl02-hgx-0057', 'slc01-cl02-hgx-0062', 'slc01-cl02-hgx-0066', 'slc01-cl02-hgx-0072', 'slc01-cl02-hgx-0073', 'slc01-cl02-hgx-0074', 'slc01-cl02-hgx-0079', 'slc01-cl02-hgx-0082', 'slc01-cl02-hgx-0083', 'slc01-cl02-hgx-0085', 'slc01-cl02-hgx-0090', 'slc01-cl02-hgx-0092', 'slc01-cl02-hgx-0099', 'slc01-cl02-hgx-0102', 'slc01-cl02-hgx-0104', 'slc01-cl02-hgx-0107', 'slc01-cl02-hgx-0112', 'slc01-cl02-hgx-0115', 'slc01-cl02-hgx-0118', 'slc01-cl02-hgx-0119', 'slc01-cl02-hgx-0130', 'slc01-cl0